In [1]:
#installing the packages
!pip install yt-dlp
!pip install openai-whisper
!pip install scenedetect
!pip install moviepy
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 9.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s et

In [2]:
#importing the libraries
import os
import sys
import subprocess
import shutil
import whisper
import moviepy.editor as mp
from scenedetect import SceneManager, VideoManager
from scenedetect.detectors import ContentDetector
import gradio as gr
#from scenedetect.video_splitter import VideoSplitter

  if event.key is 'enter':



In [ ]:
KEYWORDS = {
    'unboxing': ['unbox', 'box', 'what’s inside', 'inside the box'],
    'feature_demo': ['features', 'includes', 'it comes with', 'demo'],
    'final_verdict': ['verdict', 'final thoughts', 'should you buy', 'is it worth']
}

BUFFER_SECONDS = 20
MIN_CLIP = 30
MAX_CLIP = 60

#Downloading the video
def download_youtube_video(url, output_path='video.mp4'):
    try:
        command = [
            "yt-dlp",
            "-f", "bestvideo+bestaudio",
            "--merge-output-format", "mp4",
            "-o", output_path,
            url
        ]
        subprocess.run(command, check=True)
        print("✅ Video downloaded.")
    except subprocess.CalledProcessError as e:
        print("❌ Error downloading video:", e)
        raise

#Transcribe the audio
def transcribe_video(video_path):
    model = whisper.load_model("base")
    result = model.transcribe(video_path)
    return result['segments']

#Detecting the scenes
def detect_scenes(video_path):
    video_manager = VideoManager([video_path])
    scene_manager = SceneManager()
    scene_manager.add_detector(ContentDetector(threshold=30.0))

    video_manager.set_downscale_factor()
    video_manager.start()
    scene_manager.detect_scenes(frame_source=video_manager)

    scene_list = scene_manager.get_scene_list()
    video_manager.release()

    return [(start.get_seconds(), end.get_seconds()) for start, end in scene_list]


#finding the key moments
def find_key_moments(segments):
    key_moments = []
    for seg in segments:
        print(f"[{seg['start']:.2f}s] {seg['text']}")
        text = seg['text'].lower()
        for label, phrases in KEYWORDS.items():
            if any(p in text for p in phrases):
                key_moments.append((label, seg['start']))
    return key_moments

#Extracting the clips
def extract_clips(video_path, key_moments, output_dir='clips'):
    os.makedirs(output_dir, exist_ok=True)
    video = mp.VideoFileClip(video_path)
    clip_paths = []

    for i, (label, timestamp) in enumerate(key_moments):
        start = max(0, timestamp - BUFFER_SECONDS)
        end = min(video.duration, timestamp + BUFFER_SECONDS)
        duration = end - start
        if duration < MIN_CLIP:
            end = min(video.duration, start + MIN_CLIP)
        elif duration > MAX_CLIP:
            end = start + MAX_CLIP

        clip = video.subclip(start, end)
        clip_path = f"{output_dir}/{i+1}_{label}.mp4"
        clip.write_videofile(clip_path, codec='libx264')
        clip_paths.append(clip_path)

    return clip_paths

#Pipeline
def process_youtube_video(url):
    try:
        download_youtube_video(url, 'video.mp4')
        segments = transcribe_video('video.mp4')
        scenes = detect_scenes('video.mp4')
        key_moments = find_key_moments(segments)

        if not key_moments:
            return "❌ No key moments found.", []

        clip_paths = extract_clips('video.mp4', key_moments)
        return "✅ Clips extracted successfully!", clip_paths
    except Exception as e:
        return f"❌ Error: {str(e)}", []

#Gradio
with gr.Blocks() as demo:
    gr.Markdown("# 🎥 YouTube Highlight Extractor")

    url_input = gr.Textbox(label="YouTube Video URL", placeholder="Paste YouTube link here...")
    run_button = gr.Button("Extract Highlights")
    output_text = gr.Textbox(label="Status")
    clips_gallery = gr.Gallery(label="Generated Clips", columns=2)

    run_button.click(
        process_youtube_video,
        inputs=url_input,
        outputs=[output_text, clips_gallery]
    )

demo.launch(debug=True)




It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2f53e01f7d63e8cdc3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


✅ Video downloaded.


100%|███████████████████████████████████████| 139M/139M [00:04<00:00, 32.9MiB/s]
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

ERROR:pyscenedetect:VideoManager is deprecated and will be removed.
INFO:pyscenedetect:Loaded 1 video, framerate: 29.970 FPS, resolution: 3840 x 2160
INFO:pyscenedetect:Detecting scenes...


[0.00s]  First of all, I want to apologize.
[1.32s]  I got a throat thing going on, but look,
[3.20s]  I got a little bit of tea here, mint majesty.
[5.56s]  We're gonna need that to stay clear.
[7.28s]  We have an important video here today.
[8.92s]  This is one of those packages that I receive.
[11.48s]  It seems like yearly from China,
[13.68s]  which acts as our best indication
[15.84s]  and our hands-on experience with the next generation iPhone
[18.80s]  before the next generation iPhone exists.
[21.76s]  I believe what I have here is a 17 Pro Max.
[25.00s]  I'm probably gonna receive some more of these
[26.88s]  for different models, so make sure to subscribe
[29.60s]  so you don't miss that.
[30.44s]  I mean, it's really your best look you're gonna get
[32.48s]  at the next generation iPhone months before
[35.00s]  it actually exists in hand for people.
[37.08s]  So I'm gonna start with this box.
[38.56s]  This is the real prime time, so don't leave the video.
[41.44s]  That's 

MoviePy - Done.
Moviepy - Writing video clips/1_unboxing.mp4



Moviepy - Done !
Moviepy - video ready clips/1_unboxing.mp4


In [7]:
###displaying the video
from IPython.display import Video, display

clip_folder = "clips"
for clip_file in sorted(os.listdir(clip_folder)):
    print(f"🎬 {clip_file}")
    display(Video(os.path.join(clip_folder, clip_file)))

🎬 1_unboxing.mp4


In [ ]:
#downloading the video
from google.colab import files
files.download("clips/1_unboxing.mp4")  # Change filename as needed

In [ ]:
#using gradio for simple UI(demo purpose)
